In [4]:
# imports
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd

In [119]:
# for Testing
product_page_url ='https://www.amazon.in/ASUS-i3-1005G1-Integrated-Transparent-K413JA-EK289T/dp/B08S4TY7WL/ref=sr_1_3?crid=1H2ECW5THL6OK&dchild=1&keywords=asus+laptops&qid=1612252842&sprefix=asus+la%2Caps%2C407&sr=8-3'

In [65]:
page_content = req.get(product_page_url,headers={"User-Agent": "Requests"}).content
bsoup_object = bs(page_content)
useful_content = bsoup_object.find('div',{'id':'centerCol'})
print(get_product_name(useful_content))
print(get_product_prices(useful_content))

ASUS VivoBook Ultra K14 (2020), Intel Core i3-1005G1 10th Gen, 14-Inch FHD Thin and Light Laptop (8GB RAM/512GB SSD/Windows 10/Integrated Graphics/Transparent Silver/1.4 kg), K413JA-EK289T
[{'M.R.P.:': ' ₹\xa049,990.00'}, {'Price:': '₹\xa042,295.00'}]


In [18]:
def get_product_name(content):
    block = content.find('div',{'id':'title_feature_div'})
    if block.h1 != None :
        h1 =block.h1
        if h1.find('span') :
            return h1.find('span').text.strip()

In [112]:
def get_product_prices(content):
    block = content.find('div',{'id':'desktop_unifiedPrice'})
    try:
        if block.find('table'):
            table = block.find('table')
            table_rows = table.find_all('tr')

            prices = []
            for row in table_rows:
                 if row.find('td'):
                    row_data = row.find_all('td')
                    if len(row_data) ==2:
                        price_key =""
                        for data in row_data:
                            if data.find('span') and price_key !='':
                                 prices.append({price_key:data.span.text})   
                            else :
                                price_key = data.text
            return prices
    except : 
        return

In [66]:
def get_product_details(product_page_url):
    page_content = req.get(product_page_url,headers={"User-Agent": "Requests"}).content
    bsoup_object = bs(page_content)
    useful_content = bsoup_object.find('div',{'id':'centerCol'})
    return { get_product_name(useful_content):get_product_prices(useful_content)}

## All functions are ready for scrape specific product information 
### Let use this functions for search specific result on amazon

In [73]:
base_url = 'https://www.amazon.in'

In [68]:
def get_search_query(search):
    template = '/s?k={}'
    query = search.replace(' ','+')
    return template.format(query)

In [69]:
def filter_product_url(card):
    product = card.find('h2')
    if product != None and product.find('a'):
        url = product.find('a').get('href')
        return url

In [75]:
def get_all_products_url(url):
    page_content = req.get(url,headers={"User-Agent": "Requests"}).content
    # convert into beautiful soup object
    bsoup = bs(page_content)
    product_cards = bsoup.find_all('div' , {'class': 'sg-col-inner'})
    products = []
    for card in product_cards:
        product_name = filter_product_url(card)
        if product_name is not None:
            products.append(product_name)
    return products

In [77]:
products_pages = get_all_products_url(base_url+get_search_query("asus laptops"))

In [80]:
products_price_details =[]
for index , product in enumerate(products_pages):
    if index ==10 :
        break
    products_price_details.append(get_product_details(base_url+product))


In [99]:
def format_data(product_list):
    product_names =[]
    MRPs =[]
    prices =[]
    for product in product_list:
        name = list(product.keys())[0]
        product_names.append(name)
        details = product[name]
        mrp = details[0][list(details[0].keys())[0]]        
        price = details[1][list(details[1].keys())[0]]
        MRPs.append(mrp)
        prices.append(price)
    return {"Product Name" : product_names,"MRP" :MRPs,"Selling Price":prices}

In [107]:
formated_data = format_data(products_price_details)

In [106]:
data_frame = pd.DataFrame(formated_data)
data_frame

,Product Name,MRP,Selling Price
0,ASUS VivoBook15 X509MA-BR336T-Intel Pentium Si...,"₹ 30,990.00","₹ 28,500.00"
1,ASUS VivoBook15 X509MA-BR336T-Intel Pentium Si...,"₹ 30,990.00","₹ 28,500.00"
2,ASUS VivoBook15 X509MA-BR336T-Intel Pentium Si...,"₹ 30,990.00","₹ 28,500.00"
3,ASUS VivoBook15 X509MA-BR336T-Intel Pentium Si...,"₹ 30,990.00","₹ 28,500.00"
4,"ASUS VivoBook Ultra K14 (2020), Intel Core i3-...","₹ 52,500.00","₹ 42,250.00"
5,"ASUS VivoBook Ultra K14 (2020), Intel Core i3-...","₹ 52,500.00","₹ 42,250.00"
6,"ASUS VivoBook Ultra K14 (2020), Intel Core i3-...","₹ 52,500.00","₹ 42,250.00"
7,"ASUS VivoBook Ultra K14 (2020), Intel Core i3-...","₹ 59,500.00","₹ 43,450.00"
8,"ASUS VivoBook Ultra K14 (2020), Intel Core i3-...","₹ 59,500.00","₹ 43,450.00"
9,"ASUS VivoBook Ultra K14 (2020), Intel Core i3-...","₹ 59,500.00","₹ 43,450.00"


# Put all logic into One function 

In [108]:
def get_products_by_name(product_name):
    products_pages = get_all_products_url(base_url+get_search_query(product_name))
    products_price_details =[]
    for index , product in enumerate(products_pages):
        if index ==10 :
            break
        products_price_details.append(get_product_details(base_url+product))
    return format_data(products_price_details)

In [114]:
product_list = get_products_by_name("samsung mobiles")
data_frame = pd.DataFrame(product_list)

In [115]:
data_frame

,Product Name,MRP,Selling Price
0,"Samsung Galaxy M31 Prime Edition (Space Black,...","₹ 19,999.00","₹ 16,499.00"
1,"Samsung Galaxy M31 Prime Edition (Space Black,...","₹ 19,999.00","₹ 16,499.00"
2,"Samsung Galaxy M31 Prime Edition (Space Black,...","₹ 19,999.00","₹ 16,499.00"
3,"Samsung Galaxy M31 Prime Edition (Space Black,...","₹ 19,999.00","₹ 16,499.00"
4,"Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64...","₹ 15,999.00","₹ 13,999.00"
5,"Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64...","₹ 15,999.00","₹ 13,999.00"
6,"Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64...","₹ 15,999.00","₹ 13,999.00"
7,"Samsung Galaxy M01 Core (Black, 2GB RAM, 32GB ...","₹ 8,499.00","₹ 5,999.00"
8,"Samsung Galaxy M01 Core (Black, 2GB RAM, 32GB ...","₹ 8,499.00","₹ 5,999.00"
9,"Samsung Galaxy M01 Core (Black, 2GB RAM, 32GB ...","₹ 8,499.00","₹ 5,999.00"


In [117]:
data = get_products_by_name("asus mobiles")
data_frame_2 = pd.DataFrame(data)

In [118]:
data_frame_2

,Product Name,MRP,Selling Price
0,"(Renewed) Asus Zenfone Max M2 (Black, 3GB RAM,...","₹ 12,999.00","₹ 7,999.00"
1,"(Renewed) Asus Zenfone Max M2 (Black, 3GB RAM,...","₹ 12,999.00","₹ 7,999.00"
2,"(Renewed) Asus Zenfone Max M2 (Black, 3GB RAM,...","₹ 12,999.00","₹ 7,999.00"
3,"(Renewed) Asus Zenfone Max M2 (Black, 3GB RAM,...","₹ 12,999.00","₹ 7,999.00"
4,"(Renewed) Asus Zenfone Max M2 (Blue, 3GB RAM, ...","₹ 12,999.00","₹ 7,899.00"
5,"(Renewed) Asus Zenfone Max M2 (Blue, 3GB RAM, ...","₹ 12,999.00","₹ 7,899.00"
6,"(Renewed) Asus Zenfone Max M2 (Blue, 3GB RAM, ...","₹ 12,999.00","₹ 7,899.00"
7,(Renewed) Asus Zenfone Max Pro M1 ZB601KL-4D10...,"₹ 12,999.00","₹ 9,999.00"
8,(Renewed) Asus Zenfone Max Pro M1 ZB601KL-4D10...,"₹ 12,999.00","₹ 9,999.00"
9,(Renewed) Asus Zenfone Max Pro M1 ZB601KL-4D10...,"₹ 12,999.00","₹ 9,999.00"
